In [2]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]


startTm = datetime.datetime.now()

startDate = '20200319'
endDate = '20200320'

readPath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

thisDate = '20200320'

readPath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\equityTradeLogs'
orderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%thisDate))
for col in ['clockAtArrival', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
                'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    orderLog[col] = orderLog[col].astype('int64')

orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

targetStock = orderLog['secid'].unique()
targetStock = np.array([int(str(i)[1:]) for i in targetStock])
targetStockSZ = sorted(targetStock[targetStock < 600000])
targetStockSH = sorted(targetStock[targetStock >= 600000])

orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog['broker'] = orderLog['accCode'] // 100
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'accCode', 'secid', 'vai']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog = orderLog.copy()

orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection'].transform('nunique')
if len(orderLog[orderLog['directNum'] != 1]) > 0:
    orderLog = orderLog[orderLog['directNum'] == 1]
assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection'].nunique() == 1).all() == True)
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    a = a[a>1].reset_index()
    orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]
orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                                 np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()
placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
def getTuple(x):
    return tuple(i for i in x)
checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 4, 1, 4)]), 0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3)]), 1,
                     np.where(checkLog['updateType'] == (0, 2, 1, 3), 2,
                     np.where(checkLog['updateType'].isin([(0, 3)]), 3,
                     np.where(checkLog['updateType'].isin([(0, ), (0, 2), (0, 2, 1)]), 4, 5)))))
orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)
orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog = orderLog[orderLog['secid'] >= 2000000].reset_index(drop=True)

readPath = r'F:\orderLog\mdData'
rawMsgDataSZ = pd.read_pickle(os.path.join(readPath, 'mdLog_msg_%s.pkl'%thisDate))
orderDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '2'][['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'OrderQty', 'Price', 'cum_volume']].reset_index(drop=True)
orderDataSZ['updateType'] = 0
tradeDataSZ = pd.concat([rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'BidApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume']],
                             rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume']]], sort=False)
tradeDataSZ['ApplSeqNum'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), tradeDataSZ['OfferApplSeqNum'], tradeDataSZ['BidApplSeqNum'])
tradeDataSZ['Side'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), 2, 1)
tradeDataSZ = tradeDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradePrice', 'TradeQty', 'cum_volume']]
tradeDataSZ['updateType'] = 4
cancelDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '4'][['SecurityID', 'BidApplSeqNum', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume']].reset_index(drop=True)
cancelDataSZ['ApplSeqNum'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, cancelDataSZ['OfferApplSeqNum'], cancelDataSZ['BidApplSeqNum'])
cancelDataSZ['Side'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, 2, 1)
cancelDataSZ = cancelDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradeQty', 'cum_volume']]
cancelDataSZ['updateType'] = 3

msgDataSZ = pd.concat([orderDataSZ, tradeDataSZ, cancelDataSZ], sort=False)
del orderDataSZ
del tradeDataSZ
del cancelDataSZ
msgDataSZ = msgDataSZ.sort_values(by=['SecurityID', 'ApplSeqNum', 'sequenceNo']).reset_index(drop=True)

msgDataSZ['TradePrice'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradePrice'], 0)
msgDataSZ['TradePrice'] = msgDataSZ['TradePrice'].astype('int64')
msgDataSZ['TradeQty'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradeQty'], 0)
msgDataSZ['TradeQty'] = msgDataSZ['TradeQty'].astype('int64')
msgDataSZ['secid'] = msgDataSZ['SecurityID'] + 2000000
assert(msgDataSZ['ApplSeqNum'].max() < 1e8)
msgDataSZ['StockSeqNum'] = msgDataSZ['SecurityID']*1e8 + msgDataSZ['ApplSeqNum']
msgDataSZ['date'] = int(thisDate)
msgDataSZ['startVolume'] = msgDataSZ.groupby(['StockSeqNum'])['cum_volume'].transform('first')

startPos = orderLog[(orderLog['date'] == int(thisDate)) & (orderLog['updateType'] == 0) & (orderLog['secid'] >= 2000000) & (orderLog['isMsg'] == 1)]
startPos = startPos.drop_duplicates(subset=['date', 'secid', 'vai'])
startPos = startPos[((startPos.clock.dt.time >= datetime.time(9, 33)) & (startPos.clock.dt.time <= datetime.time(11, 30))) |\
                        ((startPos.clock.dt.time >= datetime.time(13, 3)) & (startPos.clock.dt.time <= datetime.time(14, 55)))]
startPos['SecurityID'] = startPos['secid']-2000000
startPos['orderDirection'] = np.where(startPos['orderDirection'] == 1, 1, 2)
startPos['cum_volume'] = startPos['vai']
startPos = startPos[['SecurityID', 'cum_volume', 'orderDirection', 'accCode', 'absOrderSize', 'vai', 'group']]
startPos['stockGroup'] = startPos.groupby(['accCode', 'SecurityID']).grouper.group_info[0]
startPos = startPos.sort_values(by=['SecurityID', 'cum_volume']).reset_index(drop=True)

infoData = orderLog[(orderLog['date'] == int(thisDate)) & (orderLog['group'].isin(startPos['group'].unique())) & (orderLog['updateType'].isin([0, 3, 4]))].reset_index(drop=True)
infoData['Price'] = infoData['orderPrice'].apply(lambda x: round(x*100, 0))
infoData['Price'] = infoData['Price'].astype('int64')*100
infoData['OrderQty'] = infoData['absOrderSize']
infoData['Side'] = np.where(infoData['orderDirection'] == 1, 1, 2)
infoData['TradePrice'] = np.where(infoData['updateType'] == 4, round(infoData['tradePrice']*100, 0), 0)
infoData['TradePrice'] = infoData['TradePrice'].astype('int64')*100
statusInfo = infoData.groupby(['order'])['updateType'].apply(lambda x: tuple(x)).reset_index()
statusInfo.columns = ['order', 'statusLs']
tradePriceInfo = infoData.groupby(['order'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
tradePriceInfo.columns = ['order', 'TradePriceLs']
tradeQtyInfo = infoData.groupby(['order'])['absFilledThisUpdate'].apply(lambda x: tuple(x)).reset_index()
tradeQtyInfo.columns = ['order', 'TradeQtyLs']
infoData = infoData[infoData['updateType'] == 0]
infoData = pd.merge(infoData, statusInfo, how='left', on=['order'], validate='one_to_one')
infoData = pd.merge(infoData, tradePriceInfo, how='left', on=['order'], validate='one_to_one')
infoData = pd.merge(infoData, tradeQtyInfo, how='left', on=['order'], validate='one_to_one')
infoData['brokerNum'] = infoData.groupby(['date', 'secid', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs'])['colo_broker'].transform('count')
infoData['brokerLs'] = infoData['brokerLs'] = infoData.groupby(['date', 'secid', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs'])['colo_broker'].transform(lambda x: ','.join(sorted(x.unique())))
infoData = infoData.drop_duplicates(subset=['date', 'secid', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs']).reset_index(drop=True)
infoData = infoData[['date', 'secid', 'vai','ars', 'group', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'brokerNum', 'brokerLs', 'order']]

checkLog = msgDataSZ[msgDataSZ['updateType'] == 0][['StockSeqNum', 'SecurityID', 'cum_volume', 'sequenceNo', 'clockAtArrival', 'Side', 'OrderQty', 'Price', ]].reset_index(drop=True)
checkLog = checkLog.sort_values(by=['SecurityID', 'sequenceNo'])
checkLog = pd.merge(checkLog, startPos, how='outer', on=['SecurityID', 'cum_volume'], validate='many_to_one')
del startPos
if checkLog[(~checkLog['vai'].isnull()) & (checkLog['clockAtArrival'].isnull())].shape[0] > 0:
    print('some order vai are wrong')
    removeGroup = checkLog[(~checkLog['vai'].isnull()) & (checkLog['clockAtArrival'].isnull())]['stockGroup'].unique()
    checkLog = checkLog[~checkLog['stockGroup'].isin(removeGroup)].reset_index(drop=True)    
assert(checkLog[(~checkLog['vai'].isnull()) & (checkLog['clockAtArrival'].isnull())].shape[0] == 0)


C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


some order vai are wrong


In [8]:
checkLog[checkLog["group"]==4170]

,StockSeqNum,SecurityID,cum_volume,sequenceNo,clockAtArrival,Side,OrderQty,Price,orderDirection,accCode,absOrderSize,vai,group,stockGroup
143801,205567853.0,2,23568119.0,34058164.0,1.584670e+15,1.0,100.0,254600.0,2.0,8867.0,500.0,23568119.0,4170.0,397.0
143802,205567990.0,2,23568119.0,34058853.0,1.584670e+15,1.0,100.0,254500.0,2.0,8867.0,500.0,23568119.0,4170.0,397.0
143803,205568034.0,2,23568119.0,34059051.0,1.584670e+15,1.0,300.0,254300.0,2.0,8867.0,500.0,23568119.0,4170.0,397.0
143804,205568101.0,2,23568119.0,34059797.0,1.584670e+15,1.0,2600.0,254800.0,2.0,8867.0,500.0,23568119.0,4170.0,397.0
143805,205568109.0,2,23568119.0,34059826.0,1.584670e+15,2.0,500.0,254800.0,2.0,8867.0,500.0,23568119.0,4170.0,397.0


In [9]:
checkLog['lastClockInVol'] = checkLog.groupby(['SecurityID', 'cum_volume'])['clockAtArrival'].transform('last')
checkLog['startClock'] = np.where((~checkLog['group'].isnull()) & (checkLog['clockAtArrival'] == checkLog['lastClockInVol']),
                                      checkLog['clockAtArrival'], np.nan)
checkLog['group'] = checkLog['group'].ffill()
checkLog['startClock'] = checkLog.groupby(['SecurityID', 'group'])['startClock'].transform('max')
checkLog['endClock'] = checkLog['startClock'] + 20*1e3
checkLog[checkLog["group"]==4170]

,StockSeqNum,SecurityID,cum_volume,sequenceNo,clockAtArrival,Side,OrderQty,Price,orderDirection,accCode,absOrderSize,vai,group,stockGroup,lastClockInVol,startClock,endClock
143801,205567853.0,2,23568119.0,34058164.0,1.584670e+15,1.0,100.0,254600.0,2.0,8867.0,500.0,23568119.0,4170.0,397.0,1.584670e+15,1.584670e+15,1.584670e+15
143802,205567990.0,2,23568119.0,34058853.0,1.584670e+15,1.0,100.0,254500.0,2.0,8867.0,500.0,23568119.0,4170.0,397.0,1.584670e+15,1.584670e+15,1.584670e+15
143803,205568034.0,2,23568119.0,34059051.0,1.584670e+15,1.0,300.0,254300.0,2.0,8867.0,500.0,23568119.0,4170.0,397.0,1.584670e+15,1.584670e+15,1.584670e+15
143804,205568101.0,2,23568119.0,34059797.0,1.584670e+15,1.0,2600.0,254800.0,2.0,8867.0,500.0,23568119.0,4170.0,397.0,1.584670e+15,1.584670e+15,1.584670e+15
143805,205568109.0,2,23568119.0,34059826.0,1.584670e+15,2.0,500.0,254800.0,2.0,8867.0,500.0,23568119.0,4170.0,397.0,1.584670e+15,1.584670e+15,1.584670e+15
143806,205568119.0,2,23568619.0,34059870.0,1.584670e+15,2.0,400.0,254500.0,NaN,NaN,NaN,NaN,4170.0,NaN,1.584670e+15,1.584670e+15,1.584670e+15
143807,205568149.0,2,23569019.0,34060055.0,1.584670e+15,1.0,1500.0,254700.0,NaN,NaN,NaN,NaN,4170.0,NaN,1.584670e+15,1.584670e+15,1.584670e+15
143808,205568167.0,2,23569019.0,34060145.0,1.584670e+15,2.0,1400.0,254900.0,NaN,NaN,NaN,NaN,4170.0,NaN,1.584670e+15,1.584670e+15,1.584670e+15
143809,205568220.0,2,23569019.0,34060452.0,1.584670e+15,1.0,200.0,254700.0,NaN,NaN,NaN,NaN,4170.0,NaN,1.584670e+15,1.584670e+15,1.584670e+15
143810,205568262.0,2,23569019.0,34060635.0,1.584670e+15,2.0,500.0,254500.0,NaN,NaN,NaN,NaN,4170.0,NaN,1.584670e+15,1.584670e+15,1.584670e+15


In [10]:
checkLog = checkLog[(checkLog['clockAtArrival'] >= checkLog['startClock']) & (checkLog['clockAtArrival'] <= checkLog['endClock'])]
checkLog['vai'] = checkLog.groupby(['SecurityID', 'group'])['vai'].ffill()
checkLog['orderDirection'] = checkLog.groupby(['SecurityID', 'group'])['orderDirection'].ffill()
checkLog[checkLog["group"]==4170]

,StockSeqNum,SecurityID,cum_volume,sequenceNo,clockAtArrival,Side,OrderQty,Price,orderDirection,accCode,absOrderSize,vai,group,stockGroup,lastClockInVol,startClock,endClock
143805,205568109.0,2,23568119.0,34059826.0,1.584670e+15,2.0,500.0,254800.0,2.0,8867.0,500.0,23568119.0,4170.0,397.0,1.584670e+15,1.584670e+15,1.584670e+15
143806,205568119.0,2,23568619.0,34059870.0,1.584670e+15,2.0,400.0,254500.0,2.0,NaN,NaN,23568119.0,4170.0,NaN,1.584670e+15,1.584670e+15,1.584670e+15


In [11]:
checkLog = checkLog[checkLog['Side'] == checkLog['orderDirection']]
checkLog[checkLog["group"]==4170]

,StockSeqNum,SecurityID,cum_volume,sequenceNo,clockAtArrival,Side,OrderQty,Price,orderDirection,accCode,absOrderSize,vai,group,stockGroup,lastClockInVol,startClock,endClock
143805,205568109.0,2,23568119.0,34059826.0,1.584670e+15,2.0,500.0,254800.0,2.0,8867.0,500.0,23568119.0,4170.0,397.0,1.584670e+15,1.584670e+15,1.584670e+15
143806,205568119.0,2,23568619.0,34059870.0,1.584670e+15,2.0,400.0,254500.0,2.0,NaN,NaN,23568119.0,4170.0,NaN,1.584670e+15,1.584670e+15,1.584670e+15


In [12]:
group_list = checkLog["group"].unique()
checkLog = pd.merge(msgDataSZ, checkLog[['StockSeqNum', 'vai', 'group']], how='left', on=['StockSeqNum'], validate='many_to_one')
del msgDataSZ
checkLog = checkLog[~checkLog['group'].isnull()]
display(checkLog[checkLog["group"]==4170])
statusInfo = checkLog.groupby(['StockSeqNum'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
statusInfo.columns = ['StockSeqNum', 'statusLs']
tradePriceInfo = checkLog.groupby(['StockSeqNum'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
tradePriceInfo.columns = ['StockSeqNum', 'TradePriceLs']
tradeQtyInfo = checkLog.groupby(['StockSeqNum'])['TradeQty'].apply(lambda x: tuple(x)).reset_index()
tradeQtyInfo.columns = ['StockSeqNum', 'TradeQtyLs']
checkLog = checkLog[checkLog['updateType'] == 0]
display(checkLog[checkLog["group"]==4170])
checkLog = pd.merge(checkLog, statusInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
checkLog = pd.merge(checkLog, tradePriceInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
checkLog = pd.merge(checkLog, tradeQtyInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
display(checkLog[checkLog["group"]==4170])

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,startVolume,vai,group
365757,2,5568109.0,1584669536155390,34059826,2.0,500.0,254800.0,23568119.0,0,0,0,2000002,205568109.0,20200320,23568119.0,23568119.0,4170.0
365758,2,5568109.0,1584669536155392,34059828,2.0,NaN,NaN,23568619.0,4,254800,500,2000002,205568109.0,20200320,23568119.0,23568119.0,4170.0
365759,2,5568119.0,1584669536160960,34059870,2.0,400.0,254500.0,23568619.0,0,0,0,2000002,205568119.0,20200320,23568619.0,23568119.0,4170.0
365760,2,5568119.0,1584669536160961,34059871,2.0,NaN,NaN,23569019.0,4,254800,400,2000002,205568119.0,20200320,23568619.0,23568119.0,4170.0


,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,startVolume,vai,group
365757,2,5568109.0,1584669536155390,34059826,2.0,500.0,254800.0,23568119.0,0,0,0,2000002,205568109.0,20200320,23568119.0,23568119.0,4170.0
365759,2,5568119.0,1584669536160960,34059870,2.0,400.0,254500.0,23568619.0,0,0,0,2000002,205568119.0,20200320,23568619.0,23568119.0,4170.0


,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,startVolume,vai,group,statusLs,TradePriceLs,TradeQtyLs
20,2,5568109.0,1584669536155390,34059826,2.0,500.0,254800.0,23568119.0,0,0,0,2000002,205568109.0,20200320,23568119.0,23568119.0,4170.0,"(0, 4)","(0, 254800)","(0, 500)"
21,2,5568119.0,1584669536160960,34059870,2.0,400.0,254500.0,23568619.0,0,0,0,2000002,205568119.0,20200320,23568619.0,23568119.0,4170.0,"(0, 4)","(0, 254800)","(0, 400)"


In [13]:
infoData = infoData[infoData["group"].isin(group_list)]
checkLog = pd.merge(checkLog, infoData, how='outer', on=['date', 'secid', 'group', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs'], validate='many_to_one')
display(checkLog[checkLog["group"]==4170])
checkLog = checkLog.sort_values(by=['date', 'secid', 'vai', 'sequenceNo']).reset_index(drop=True)
checkLog['groupPrice'] = checkLog.groupby(['group'])['Price'].transform('median')
checkLog['Price'] = np.where(checkLog['Price'] == 0, checkLog['groupPrice'], checkLog['Price'])
checkLog['OrderNtl'] = checkLog['Price'] * checkLog['OrderQty'] / 10000

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,startVolume,vai,group,statusLs,TradePriceLs,TradeQtyLs,ars,brokerNum,brokerLs,order
20,2.0,5568109.0,1.584670e+15,34059826.0,2.0,500.0,254800.0,23568119.0,0.0,0.0,0.0,2000002,205568109.0,20200320,23568119.0,23568119.0,4170.0,"(0, 4)","(0, 254800)","(0, 500)",11.0,1.0,zs_88,5509.0
21,2.0,5568119.0,1.584670e+15,34059870.0,2.0,400.0,254500.0,23568619.0,0.0,0.0,0.0,2000002,205568119.0,20200320,23568619.0,23568119.0,4170.0,"(0, 4)","(0, 254800)","(0, 400)",NaN,NaN,NaN,NaN


In [38]:
checkLog[checkLog["group"] == 4155]

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,startVolume,vai,group,statusLs,TradePriceLs,TradeQtyLs,ars,brokerNum,brokerLs,order,groupPrice,OrderNtl
0,1.0,10015541.0,1.584673e+15,62526298.0,2.0,100.0,125000.0,78530508.0,0.0,0.0,0.0,2000001,110015541.0,20200320,78530508.0,78530508.0,4155.0,"(0, 4)","(0, 125000)","(0, 100)",NaN,NaN,NaN,NaN,125000.0,1250.0
1,1.0,10015544.0,1.584673e+15,62526303.0,2.0,100.0,125000.0,78530608.0,0.0,0.0,0.0,2000001,110015544.0,20200320,78530608.0,78530508.0,4155.0,"(0, 4)","(0, 125000)","(0, 100)",NaN,NaN,NaN,NaN,125000.0,1250.0
2,1.0,10015551.0,1.584673e+15,62526320.0,2.0,9100.0,125000.0,78530708.0,0.0,0.0,0.0,2000001,110015551.0,20200320,78530708.0,78530508.0,4155.0,"(0, 4)","(0, 125000)","(0, 9100)",11.0,1.0,zs_88,5498.0,125000.0,113750.0
